<a href="https://colab.research.google.com/github/nijatmaharramov/Computer-Vision-Projects/blob/main/Facial_emotion_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d sujaykapadnis/emotion-recognition-dataset

In [ ]:
import zipfile

with zipfile.ZipFile('/content/emotion-recognition-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
import shutil

shutil.rmtree('/content/dataset/Ahegao')

In [ ]:
import os

for dirpath, dirnames, filenames, in os.walk('/content/dataset'):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

In [ ]:
import tensorflow as tf

IMG_SIZE = (224, 224)

data_path = '/content/dataset'

train_data = tf.keras.preprocessing.image_dataset_from_directory(data_path,
                                                                 label_mode = 'categorical',
                                                                 image_size = IMG_SIZE,
                                                                 validation_split = 0.2,
                                                                 subset = 'training',
                                                                 seed = 1)

test_data = tf.keras.preprocessing.image_dataset_from_directory(data_path,
                                                                label_mode = 'categorical',
                                                                image_size = IMG_SIZE,
                                                                validation_split = 0.2,
                                                                subset = 'validation',
                                                                seed = 1)

In [ ]:
class_names = train_data.class_names
class_names

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# class_names = train_data.class_names

plt.figure(figsize = (12,10))
for images, labels in train_data.take(1):
    for i in range(12):
        plt.subplot(3, 4, i + 1)
        plt.imshow(images[i] / 255)
        plt.title(class_names[np.argmax(labels[i].numpy())])
        plt.axis(False)

In [ ]:
# train_data = train_data.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
# test_data = test_data.cache().prefetch(buffer_size = tf.data.AUTOTUNE)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape = (224, 224, 3)),
    tf.keras.layers.Rescaling(1 / 255.),
    tf.keras.layers.Conv2D(64, kernel_size = 7, strides = 3, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, kernel_size = 3, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal'),
    tf.keras.layers.Conv2D(128, kernel_size = 3, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(256, kernel_size = 3, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal'),
    tf.keras.layers.Conv2D(256, kernel_size = 3, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation = 'relu', kernel_initializer = 'he_normal'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation = 'relu', kernel_initializer = 'he_normal'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(class_names), activation = 'softmax')
                            ])

In [ ]:
model.summary()

In [ ]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(),
              metrics = ['accuracy', 'f1_score'])

model.fit(train_data, epochs = 5, validation_data = test_data)

# Resnet50

In [ ]:
inputs = tf.keras.layers.Input(shape = (224, 224, 3))

x = tf.keras.applications.resnet50.preprocess_input(inputs)

base_model = tf.keras.applications.ResNet50(include_top = False, input_tensor = x)

avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

output = tf.keras.layers.Dense(len(class_names), activation = 'softmax')(avg)
model2 = tf.keras.Model(inputs = inputs, outputs = output)

In [ ]:
base_model.trainable = False

In [ ]:
initial_learning_rate = 0.01
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps = 357,
    decay_rate = 0.96
                                                        )

early_stopping = tf.keras.callbacks.EarlyStopping(patience = 2,
                                                  restore_best_weights = True)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    'emotion_detector.keras',
    save_best_only = True
)

model2.compile(
    loss = 'categorical_crossentropy',
    optimizer = tf.keras.optimizers.Adam(lr_schedule),
    metrics = ['accuracy', 'f1_score']
            )

model2.fit(train_data, epochs = 10, validation_data = test_data,
          callbacks = [early_stopping, checkpoint_cb])

# Predictive system

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def plot_and_pred(model, image_path):
    plt.figure(figsize = (10,8))
    image = mpimg.imread(image_path)
    image = tf.image.resize(image, (224, 224))
    pred_probs = model.predict(np.expand_dims(image, axis=0))
    pred = class_names[np.argmax(pred_probs)]
    plt.subplot(1, 2, 1)
    plt.imshow(image / 255.)
    plt.title(pred, color = 'green')
    plt.axis('off')
    plt.subplot(1, 2, 2)
    plt.bar(class_names, pred_probs[0])
    plt.xlabel('Class')
    plt.ylabel('Probability')

In [ ]:
# plot_and_pred(model2, '/content/Happiness-1-506x319.png')

# Xception

In [ ]:
inputs = tf.keras.layers.Input(shape = (224, 224, 3))

x = tf.keras.applications.xception.preprocess_input(inputs)

base_model3 = tf.keras.applications.Xception(include_top = False, input_tensor = x)

avg = tf.keras.layers.GlobalAveragePooling2D()(base_model3.output)

output = tf.keras.layers.Dense(len(class_names), activation = 'softmax')(avg)

model3 = tf.keras.Model(inputs=inputs, outputs=output)



In [ ]:
base_model3.trainable = False

In [ ]:
initial_lr = 0.01
schedule_lr = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_lr,
    decay_steps = 357,
    decay_rate = 0.96
)

early_bird = tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)


# ne oldugun sorus
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    'emotion_detector.keras',
    save_best_only = True
)

model3.compile(loss='categorical_crossentropy',
               optimizer=tf.keras.optimizers.Adam(learning_rate = schedule_lr),
               metrics = ['accuracy', 'f1_score'])

model3.fit(train_data, epochs = 10, validation_data = test_data,
           callbacks = [early_bird, checkpoint_cb])
